<div class='bar_title'></div>

*Decision Support Systems*

# Bonus 01

Toni Greif <br>

Winter Semester 19/20

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Standard-Model" data-toc-modified-id="Standard-Model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Standard Model</a></span></li><li><span><a href="#Alternative-Model" data-toc-modified-id="Alternative-Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Alternative Model</a></span></li><li><span><a href="#Continued-Model" data-toc-modified-id="Continued-Model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Continued Model</a></span></li></ul></div>

In [1]:
!pip install pulp
from pulp import *
import pandas as pd

## Standard Model

In [2]:
# Models
m1 = LpProblem("Wildlife", sense=pulp.LpMaximize)
m2 = m1.copy()
m3 = m1.copy()
m4 = m1.copy()
m5 = m1.copy()
m6 = m1.copy()

models = [m1,m2,m3,m4,m5,m6]

In [3]:
# Dictionaries
coeff = {'Elephant': {'B': 1800, 'C': 0, 'T': 0, 'L': 0, 'S': 2200, 'y': 0.04},
        'WhiteRhino': {'B': 1500, 'C': 2000, 'T': 8500, 'L': 2000, 'S': 0, 'y': 0.08},
        'BlackRhino': {'B': 860, 'C': 15000, 'T': 0, 'L': 0, 'S': 0, 'y': 0.08},
        'Giraffe': {'B': 750, 'C': 0, 'T': 0, 'L': 2000, 'S': 920, 'y': 0.10},
        'Buffalo': {'B': 450, 'C': 4000, 'T': 0, 'L': 0, 'S': 0, 'y': 0.10},
        'Eland': {'B': 360, 'C': 0, 'T': 0, 'L': 500, 'S': 440, 'y': 0.15},
        'Zebra': {'B': 210, 'C': 0, 'T': 450, 'L': 220, 'S': 250, 'y': 0.10},
        'Roan': {'B': 200, 'C': 6000, 'T': 0, 'L': 0, 'S': 0, 'y': 0.10}
        }

In [4]:
consum = {'Elephant': {'Browse2.5': 0, 'Browse5': 20.57, 'Forbs': 1.21, 'Leaf': 0.48, 'GrasFine': 1.21, 'GrasCoarse': 0.73},
        'WhiteRhino': {'Browse2.5': 0, 'Browse5': 0, 'Forbs': 1.03, 'Leaf': 0, 'GrasFine': 11.33, 'GrasCoarse': 8.24},
        'BlackRhino': {'Browse2.5': 12.66, 'Browse5': 12.66, 'Forbs': 1.49, 'Leaf': 0.45, 'GrasFine': 0, 'GrasCoarse': 0.3},
        'Giraffe': {'Browse2.5': 0, 'Browse5': 13.72, 'Forbs': 0.14, 'Leaf': 0.14, 'GrasFine': 0, 'GrasCoarse': 0},
        'Buffalo': {'Browse2.5': 0, 'Browse5': 0, 'Forbs': 0.09, 'Leaf': 0.09, 'GrasFine': 5.16, 'GrasCoarse': 3.56},
        'Eland': {'Browse2.5': 6.2, 'Browse5': 6.2, 'Forbs': 0.73, 'Leaf': 0, 'GrasFine': 0.22, 'GrasCoarse': 0.15},
        'Zebra': {'Browse2.5': 0, 'Browse5': 0, 'Forbs': 0, 'Leaf': 0, 'GrasFine': 3.13, 'GrasCoarse': 2.57},
        'Roan': {'Browse2.5': 0.37, 'Browse5': 0.37, 'Forbs': 0, 'Leaf': 0, 'GrasFine': 2.56, 'GrasCoarse': 1.57}
         }

In [5]:
biomass = {'Browse2.5': {1: 81940, 2: 152180, 3: 751759},
           'Browse5': {1: 163331, 2: 366436, 3: 1642627},
           'Forbs': {1: 171990, 2: 140934, 3: 251154},
           'Leaf': {1: 626592, 2: 320736, 3: 1853478},
           'GrasFine': {1: 468406, 2: 439470, 3: 1461834},
           'GrasCoarse': {1: 506617, 2: 536616, 3: 1786989},
          }

In [6]:
# Sets
species = ['Elephant', 'WhiteRhino', 'BlackRhino', 'Giraffe', 'Buffalo', 'Eland', 'Zebra', 'Roan']
habitats = [1, 2, 3]
foods = biomass.keys()
values = ['C', 'T', 'L','S']

We introduce variables for the number of animals per habitat and for the animals used for the respective sales objectives.

In [7]:
# Variables 
x = LpVariable.dicts("x", (species, habitats), lowBound=0, cat='Continuous')
y = LpVariable.dicts("y", (species, values), lowBound=0, cat='Continuous')

This allows the objective functions to be written very intuitively.

In [8]:
# Objectives
m1 += lpSum([coeff[s]['B'] * x[s][h] for s in species for h in habitats])
m2 += lpSum([x[s][h] for s in species for h in habitats])
m3 += lpSum([coeff[s]['C'] * y[s]['C'] for s in species])
m4 += lpSum([coeff[s]['T'] * y[s]['T'] for s in species])
m5 += lpSum([coeff[s]['L'] * y[s]['L'] for s in species])
m6 += lpSum([coeff[s]['S'] * y[s]['S'] for s in species])

The amount of food eaten by the animals in each habitat is constraint to the food available in the habitat. Depending on the objective function the number of animals used has to be constrained to the yearly increase of that species.

In [9]:
# Constraints
## Amount of food
for f in foods:  
    for h in habitats:
        for m in models:
            m += lpSum([consum[s][f] * x[s][h] for s in species]) <= (biomass[f][h]/100)
        
## Animals used      
for s in species:
        ## Conservation
        m3 += lpSum(coeff[s]['y'] * x[s][h] for h in habitats) >= y[s]['C']
        ## Trophies
        m4 += lpSum(coeff[s]['y'] * x[s][h] for h in habitats) >= y[s]['T']
        ## Alive
        m5 += lpSum(coeff[s]['y'] * x[s][h] for h in habitats) >= y[s]['L']
        # Carcasses 
        m6 += lpSum(coeff[s]['y'] * x[s][h] for h in habitats) >= y[s]['S']

We solve all models and print the results.

In [10]:
# Solve ALL models
for m in models:
    m.solve()
    print (LpStatus[m.status], ': Objective=',value(m.objective))
    print(pd.DataFrame([[x[s][h].varValue for h in habitats] for s in species], columns = habitats, index = species))
    print('\n')
    
print(pd.DataFrame([[y[s][v].varValue for v in values] for s in species], columns=values, index = species))

Optimal : Objective= 4869099.6263999995
                     1          2           3
Elephant     79.402528  178.14098   798.55469
WhiteRhino  404.941120  368.85696  1204.95050
BlackRhino    0.000000    0.00000     0.00000
Giraffe       0.000000    0.00000     0.00000
Buffalo       0.000000    0.00000     0.00000
Eland         0.000000    0.00000     0.00000
Zebra         0.000000    0.00000     0.00000
Roan          0.000000    0.00000     0.00000


Optimal : Objective= 11074.910493
                      1           2           3
Elephant       0.000000     0.00000     0.00000
WhiteRhino     0.000000     0.00000     0.00000
BlackRhino     0.000000     0.00000     0.00000
Giraffe       59.322886   156.16327   649.32070
Buffalo        0.000000     0.00000     0.00000
Eland         23.087267   143.74178   876.23299
Zebra          0.000000     0.00000     0.00000
Roan        1827.726900  1704.32690  5634.98780


Optimal : Objective= 6163848.435
                      1            2       

## Alternative Model

We do not necessarily have to introduce a variable for the animals used, alternatively it can also be modeled via the objective function.

In [11]:
# Model
m5_v2 = LpProblem("Wildlife", sense=pulp.LpMaximize)

We include the yield directly in the objective function and use the ``x`` variables from before.

In [12]:
print(x)

{'Elephant': {1: x_Elephant_1, 2: x_Elephant_2, 3: x_Elephant_3}, 'WhiteRhino': {1: x_WhiteRhino_1, 2: x_WhiteRhino_2, 3: x_WhiteRhino_3}, 'BlackRhino': {1: x_BlackRhino_1, 2: x_BlackRhino_2, 3: x_BlackRhino_3}, 'Giraffe': {1: x_Giraffe_1, 2: x_Giraffe_2, 3: x_Giraffe_3}, 'Buffalo': {1: x_Buffalo_1, 2: x_Buffalo_2, 3: x_Buffalo_3}, 'Eland': {1: x_Eland_1, 2: x_Eland_2, 3: x_Eland_3}, 'Zebra': {1: x_Zebra_1, 2: x_Zebra_2, 3: x_Zebra_3}, 'Roan': {1: x_Roan_1, 2: x_Roan_2, 3: x_Roan_3}}


In [13]:
# Objective
m5_v2 += lpSum([coeff[s]['L'] * coeff[s]['y'] * x[s][h] for s in species for h in habitats]) 

The constraints simplify significantly.

In [14]:
# Constraints
## Amount of food
for f in foods:  
    for h in habitats:
            m5_v2 += lpSum([consum[s][f] * x[s][h] for s in species]) <= (biomass[f][h]/100)

In [15]:
# Solve problem
m5_v2.solve()
print (LpStatus[m5_v2.status], ': Objective=',value(m5_v2.objective))
print(pd.DataFrame([[x[s][h].varValue for h in habitats] for s in species], columns = habitats, index = species))

Optimal : Objective= 651321.2284
                    1          2         3
Elephant      0.00000    0.00000     0.000
WhiteRhino  413.42101  387.88173  1290.233
BlackRhino    0.00000    0.00000     0.000
Giraffe     119.04592  267.08163  1197.250
Buffalo       0.00000    0.00000     0.000
Eland         0.00000    0.00000     0.000
Zebra         0.00000    0.00000     0.000
Roan          0.00000    0.00000     0.000


 Compare the solution with the one of model m5! <img src="images/UpToYou.png" style="width:8%; float:left">

## Continued Model

Given the capture objective, the prices for animals fall if the supply gets to big. We assume that the buyers for white rhinos as well as giraffes are willing to buy 100 animals of both species for the old price. If the park management wants to sell more of these animals the price for each additional animal is 20% lower.

To model this situation we extend model m5 and introduce two new species ``WhiteRhinoDisc`` and ``GiraffeDisc``.
The live coefficienten reduces to 1600.

In [16]:
species_disc = ['WhiteRhino','Giraffe']
species_new = [(s+'Disc') for s in species_disc]
print(species_new)

['WhiteRhinoDisc', 'GiraffeDisc']


In [17]:
coeff = {'WhiteRhinoDisc': {'L': 1600},
        'GiraffeDisc': {'L': 1600},
        'Elephant': {'B': 1800, 'C': 0, 'T': 0, 'L': 0, 'S': 2200, 'y': 0.04},
        'WhiteRhino': {'B': 1500, 'C': 2000, 'T': 8500, 'L': 2000, 'S': 0, 'y': 0.08},
        'BlackRhino': {'B': 860, 'C': 15000, 'T': 0, 'L': 0, 'S': 0, 'y': 0.08},
        'Giraffe': {'B': 750, 'C': 0, 'T': 0, 'L': 2000, 'S': 920, 'y': 0.10},
        'Buffalo': {'B': 450, 'C': 4000, 'T': 0, 'L': 0, 'S': 0, 'y': 0.10},
        'Eland': {'B': 360, 'C': 0, 'T': 0, 'L': 500, 'S': 440, 'y': 0.15},
        'Zebra': {'B': 210, 'C': 0, 'T': 450, 'L': 220, 'S': 250, 'y': 0.10},
        'Roan': {'B': 200, 'C': 6000, 'T': 0, 'L': 0, 'S': 0, 'y': 0.10}
        }

In [18]:
# Create model
m_cont = LpProblem("Wildlife", sense=pulp.LpMaximize)

We use the same objective and variables, but with additional indices for the animals used at the discount rate.

In [19]:
# Variables
x = LpVariable.dicts("x", (species, habitats), lowBound=0, cat='Continuous')
y = LpVariable.dicts("y", ((species + species_new), values), lowBound=0, cat='Continuous')

In [20]:
# Objective 
m_cont += lpSum([coeff[s]['L'] * y[s]['L'] for s in (species + species_new)])

The food restriction remains the same as we do not differentiate the animals according to the sales price. The number of animals used at the regular rate is limited to 100 and the amount of discount and regular animals must be less than the number of available animals.

In [21]:
# Constraints
for f in foods:  
    for h in habitats:
        m_cont += lpSum([consum[s][f] * x[s][h] for s in species]) <= (biomass[f][h]/100)
            
for s in species:
    if s in species_disc:
        m_cont += lpSum(coeff[s]['y'] * x[s][h] for h in habitats) >= (y[(s+'Disc')]['L'] + y[s]['L'])
        m_cont += y[s]['L'] <= 100
    else:
        m_cont += lpSum(coeff[s]['y'] * x[s][h] for h in habitats) >= y[s]['L']

In [22]:
# Solve problem
m_cont.solve()
print (LpStatus[m_cont.status], ': Objective=',value(m_cont.objective))
print(pd.DataFrame([[x[s][h].varValue for h in habitats] for s in species], columns = habitats, index = species))
    
print(pd.DataFrame([[y[s]['L'].varValue] for s in (species+species_new)], columns=['L'], index = (species+species_new)))

Optimal : Objective= 601329.8358
                     1          2           3
Elephant      0.000000    0.00000     0.00000
WhiteRhino  410.854770  383.11568  1272.49820
BlackRhino    0.000000    0.00000     0.00000
Giraffe      59.322886  156.16327   784.51385
Buffalo       0.000000    0.00000     0.00000
Eland       132.161290  245.45161   913.34516
Zebra         0.000000    0.00000     0.00000
Roan          0.000000    0.00000     0.00000
                         L
Elephant          0.000000
WhiteRhino      100.000000
BlackRhino        0.000000
Giraffe         100.000000
Buffalo           0.000000
Eland           193.643710
Zebra             0.000000
Roan              0.000000
WhiteRhinoDisc   65.317488
GiraffeDisc       0.000000


Compare the solution with the one of model m5! <img src="images/UpToYou.png" style="width:8%; float:left">